#Environment Setting

In [0]:

import pandas as pd
import os
import datetime
#####################################################################################
################################### ONE OFF SETUP ###################################
#####################################################################################
#Mounting the blob storage for dengue documents
storage_name="xxxx"
container_name="xxxx"
blobkey="xxxx"

blobconf = "fs.azure.account.key."+storage_name+".blob.core.windows.net"
if not any(mount.mountPoint == "/mnt/"+storage_name+ "/" +container_name for mount in dbutils.fs.mounts()):
    print('nothing mounted')
    try:
        dbutils.fs.mount(
          source = "wasbs://"+container_name+"@"+storage_name+".blob.core.windows.net",
          mount_point = "/mnt/"+storage_name+ "/" +container_name,
          extra_configs = {blobconf:blobkey})
    except Exception as e:
        print("already mounted. Try to unmount first")

PATH = '/dbfs/mnt/'+storage_name+'/'+container_name +'/'


# Adoption Rate

##Data Input

##MV_portal _report

In [0]:
def prep(df):
    try:
        df.drop('Unnamed: 0',axis=1,inplace=True)
    except:
        pass
    #df = df.drop_duplicates(['Request Number'], keep='last').copy()
    df['Vehicle License Number'] = df['Vehicle License Number'].apply(lambda x: str(x).replace(" ","").lower())
    df['Request Number'] = df['Request Number'].apply(lambda x: str(x).lower())
    df['Partner Name'] = df['Partner Name'].apply(lambda x: str(x).lower().strip())
    df = df[['Request Number','Vehicle License Number','Partner Name','Partner Type','Condition','Submitted Date Time','Created Date Time','Re-submitted Date Time','Submitted By','Status']].copy()
    df['Submitted By'] = df['Submitted By'].astype(str)
    df['Submitted By'] = df['Submitted By'].apply(lambda x: str(x).lower())

    return df
def type_with_sso(row):
    row['Partner Name'] = row['Partner Name'].lower()

    if row['Partner Type'] == 'Partner' or 'cermati' in row['Partner Name']:
        return 'Partner'
    else:
        return 'Agent'


def pre_condition(df):
    # Count USED vehicles
    # Remove TBA Vehicles
    # Using Re-submitted date time as the claim survey starting date
    df = df[df['Condition']=="USED"].copy()
    df = df[~df['Vehicle License Number'].apply(lambda x: 'tba' in x)].reset_index()
    # df.loc[df['Re-submitted Date Time'].notnull(),'Submitted Date Time'] = df['Re-submitted Date Time']
    
    # #df.loc[df['Re-submitted Date Time'] != '','Submitted Date Time'] = df['Re-submitted Date Time'].values[df['Re-submitted Date Time'] != '']
    # df['Submitted Date Time'].fillna(df['Re-submitted Date Time'],inplace=True)
    # df['Submitted Date Time'].fillna(df['Created Date Time'],inplace=True)
    # df['Submitted Date Time'] = pd.to_datetime(df['Submitted Date Time'], format='%Y-%m-%d %H:%M:%S', errors='ignore')


    #df.loc[df['Re-submitted Date Time'] != '','Submitted Date Time'] = df['Re-submitted Date Time'].values[df['Re-submitted Date Time'] != '']
    df['Created Date Time'].fillna(df['Submitted Date Time'],inplace=True)
    #df['Submitted Date Time'].fillna(df['Created Date Time'],inplace=True)
    df['Created Date Time'] = pd.to_datetime(df['Created Date Time'], format='%Y-%m-%d %H:%M:%S', errors='ignore')


    df = df.drop_duplicates(subset='Created Date Time', keep='last')
    df['week'] = pd.to_datetime(df['Created Date Time']).apply(lambda x: x.strftime('%U'))
    df['month'] = pd.to_datetime(df['Created Date Time']).apply(lambda x: x.strftime('%m'))
    df = df.reset_index()
    df['Type'] = df.apply(type_with_sso,axis=1)
    # df = df.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Request Number'], keep='first')
    

    return df

  

df_mv_portal = pd.read_csv(PATH2 + 'df_mv_master_1.csv')
df_mv_portal = prep(df_mv_portal)
df_mv_portal = pre_condition(df_mv_portal)

# df_mv_portal_partner = partner(df_mv_portal)
# print(len(df_mv_portal_partner['Request Number'].unique()))
# df_mv_portal_agent = agent(df_mv_portal)
# print(len(df_mv_portal_agent['Request Number'].unique()))
# df_mv_portal_sso = sso(df_mv_portal)
# print(len(df_mv_portal_sso['Request Number'].unique()))

##Camcom_data_input

In [0]:
#Input from CamCom
df_camcom_review_completed = pd.read_csv(PATH1 + 'df_camcom_review_completed.csv')
df_camcom_review_completed = df_camcom_review_completed.ffill()
df_camcom_review_completed['Inspection ID'] = df_camcom_review_completed['Inspection ID'].apply(lambda x: x.lower())
df_camcom_review_completed['Reviewed Date'] = df_camcom_review_completed['Review Completed Date'] #update reviewed date with latest date
df_camcom_review_completed.drop(['Review Completed Date'],axis=1,inplace=True)
print("num of review_completed cases: {}".format(len(df_camcom_review_completed['Inspection ID'].unique())+5))
df_camcom_review_completed_filter = df_camcom_review_completed[['Inspection ID','Reviewed Date','No of Failed Images']].copy()

df_camcom_completed = pd.read_csv(PATH1 + 'df_camcom_completed.csv')
df_camcom_completed = df_camcom_completed.ffill()
df_camcom_completed['Inspection ID'] = df_camcom_completed['Inspection ID'].apply(lambda x: x.lower())
df_camcom_completed.drop(df_camcom_completed[df_camcom_completed['Inspection ID'].isin(df_camcom_review_completed['Inspection ID'])].index, inplace = True)
print("number of completed cases: {}".format(len(df_camcom_completed['Inspection ID'].unique())+1))
df_camcom_completed_filter = df_camcom_completed[['Inspection ID','Reviewed Date','No of Failed Images']].copy()

#df_camcom_review_failed.columns=['Inspeciton ID','Review Date','Status']
df_camcom_review_failed = pd.read_csv(PATH1 + 'camcom_failed_cases.csv')
# df_camcom_review_failed['Inspection ID'] = df_camcom_review_failed['Inspection ID'].apply(lambda x: x.split(".")[0])
df_camcom_review_failed['FileName'] = df_camcom_review_failed['FileName'].apply(lambda x: x.lower().split('.')[0])
df_camcom_review_failed['FileName'] = df_camcom_review_failed['FileName'].astype(str)

df_camcom_review_failed = df_camcom_review_failed.ffill()
df_camcom_review_failed.columns = ['Inspection ID','Reviewed Date']
print("number of failed cases: {}".format(len(df_camcom_review_failed['Inspection ID'].unique())))


#Preprocessing
df_camcom = pd.concat([df_camcom_completed_filter,df_camcom_review_completed_filter,df_camcom_review_failed], ignore_index = True)
df_camcom['Inspection ID'] = df_camcom['Inspection ID'].apply(lambda x: x.lower())
# df_camcom["Intensity"] = df_camcom["Intensity"].replace('-','0').fillna(0)
# df_camcom["Intensity"] = df_camcom["Intensity"].astype(float).astype('int')
print("number of total camcom cases: {}".format(len(df_camcom['Inspection ID'].unique())+6))


##Data Preprocessing

In [0]:

# df_mv_portal = pd.concat([df_mv_portal_agent,df_mv_portal_partner,df_mv_portal_sso],ignore_index=True)

a = df_camcom[df_camcom['Inspection ID'].isin(df_mv_portal['Vehicle License Number'])].reset_index(drop=True).copy()
b = df_camcom[df_camcom['Inspection ID'].isin(df_mv_portal['Request Number'])].reset_index(drop=True).copy()
for i,j in enumerate(b['Inspection ID']):
    index = df_mv_portal[df_mv_portal['Request Number'] == j].index[0]
    b['Inspection ID'][i] = df_mv_portal['Vehicle License Number'][index]

df_camcom_available = pd.concat([a,b],ignore_index=True)
df_camcom_available = df_camcom_available.drop_duplicates(subset='Reviewed Date', keep='last')
temp = df_mv_portal[['week','month','Vehicle License Number','Partner Name','Type']].copy()
df_camcom_available = df_camcom_available.merge(temp,left_on='Inspection ID', right_on='Vehicle License Number', how='left')
print(len(df_camcom_available['Inspection ID'].unique()))
######################################################################################################

a = df_mv_portal[df_mv_portal['Request Number'].isin(df_camcom['Inspection ID'])].reset_index(drop=True)

temp = df_mv_portal[~df_mv_portal['Request Number'].isin(df_camcom['Inspection ID'])].reset_index(drop=True)
b = temp[temp['Vehicle License Number'].isin(df_camcom['Inspection ID'])].reset_index(drop=True)

df_mv_portal_available = pd.concat([a,b],ignore_index=True) # contain duplicate vehicle license number
# df_mv_portal_available = df_mv_portal_available.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Request Number'], keep='last')
# print(len(df_mv_portal_available['Vehicle License Number'].unique()))

# df_mv_portal_available = df_mv_portal_available.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Vehicle License Number'], keep='last')
# df_mv_portal_available = df_mv_portal_available.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Request Number'], keep='last')

# #df_mv_portal = df_mv_portal[df_mv_portal['Condition']=="USED"]
# df_mv_portal = df_mv_portal[~df_mv_portal['Status'].isin(['Deleted','Canceled','Back Order','Pending'])]
# df_mv_portal = df_mv_portal[df_mv_portal['Partner Type'].isin(['Agent','Partner'])]

# df_mv_portal = df_mv_portal.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Vehicle License Number'], keep='last')
# df_mv_portal = df_mv_portal.sort_values(by=['Submitted Date Time']).drop_duplicates(subset=['Request Number'], keep='last')
df_mv_portal_available = df_mv_portal_available.sort_values(by=['Created Date Time']).drop_duplicates(subset=['Request Number'], keep='last')
print(len(df_mv_portal_available['Vehicle License Number'].unique()))

df_mv_portal_available = df_mv_portal_available.sort_values(by=['Created Date Time']).drop_duplicates(subset=['Vehicle License Number'], keep='last')
df_mv_portal_available = df_mv_portal_available.sort_values(by=['Created Date Time']).drop_duplicates(subset=['Request Number'], keep='last')

#df_mv_portal = df_mv_portal[df_mv_portal['Condition']=="USED"]
df_mv_portal = df_mv_portal[~df_mv_portal['Status'].isin(['Deleted','Canceled','Back Order','Pending'])]
df_mv_portal = df_mv_portal[df_mv_portal['Partner Type'].isin(['Agent','Partner'])]

df_mv_portal = df_mv_portal.sort_values(by=['Created Date Time']).drop_duplicates(subset=['Vehicle License Number'], keep='last')
df_mv_portal = df_mv_portal.sort_values(by=['Created Date Time']).drop_duplicates(subset=['Request Number'], keep='last')


df_mv_portal = pd.concat([df_mv_portal,df_mv_portal_available])
print(len(df_mv_portal))
print(len(df_mv_portal['Vehicle License Number'].unique()))


##TAT_ZERO_RATE

In [0]:
df_camcom_review_completed = pd.read_csv(PATH1 + 'df_camcom_review_completed.csv')
a = df_camcom_review_completed.drop_duplicates(subset='Inspection ID')

num_zeros = a['TAT_v2'].eq(0).sum()
total_values = a['TAT_v2'].notna().sum()

zero_rate = num_zeros / total_values
zero_rate

In [0]:
a['Reviewed Date'] = pd.to_datetime(a['Reviewed Date'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
a['month'] = pd.to_datetime(a['Reviewed Date']).apply(lambda x: x.strftime('%m'))
a = a.dropna(subset=['TAT_v2']).copy()
a = a[['month','TAT_v2']].copy()

# Group by 'Month' and aggregate the counts

new_df = a.groupby('month').agg({'TAT_v2': ['size', lambda x: (x == 0).sum()]})

# Flatten the multi-level column index
new_df.columns = new_df.columns.droplevel(0)
new_df = new_df.reset_index()

new_df.columns = ['Month', 'Total Cases from CamCom', 'TAT<1day']
new_df['Rate'] = new_df['TAT<1day'].astype(int) / new_df['Total Cases from CamCom'].astype(int)
new_df['Rate'] = new_df['Rate'].map('{:.2%}'.format)
print(new_df)

##Adoption rate

### Total Adoption Rate

In [0]:
#df_mv_portal_Feb = df_mv_portal[pd.to_datetime(df_mv_portal['Created Date Time']).dt.month>1]

Adoption_Rate = str(round((len(df_camcom_available['Inspection ID'].unique()) / len(df_mv_portal['Vehicle License Number'].unique()))*100,2))+"%"
Adoption_Rate = pd.DataFrame({'Adoption_Rate':[Adoption_Rate]})
Adoption_Rate

###Monthly adoption rate___ month_rate

In [0]:
month_camcom = df_camcom_available.drop_duplicates(subset=['Inspection ID'], keep='first')
df1_counts = month_camcom.groupby('month').nunique()['Inspection ID']
df2_counts = df_mv_portal.groupby('month').nunique()['Request Number']
month_df = pd.merge(df1_counts, df2_counts, on='month')
month_df['month_rate'] = month_df['Inspection ID'] / month_df['Request Number']
month_df = month_df.reset_index()
month_df['month_rate'] = month_df['month_rate'].apply(lambda x: str(round(x*100,2)) + "%")
month_df.columns = ['month','Num_Camcom','Num_MV','month_rate']
# Print the resulting dataframe

month_rate = month_df.copy()
month_rate

###Weekly adoption rate___ week_rate

In [0]:
week_camcom = df_camcom_available.drop_duplicates(subset=['Inspection ID'], keep='first')

df1_counts = week_camcom.groupby('week').nunique()['Inspection ID']
df2_counts = df_mv_portal.groupby('week').nunique()['Request Number']

# Merge the two dataframes on week
week_df = pd.merge(df1_counts, df2_counts, on='week')

# Calculate the week rate as the ratio of df1's IDs to df2's IDs
week_df['week_rate'] = week_df['Inspection ID'] / week_df['Request Number']

# Reset the index to include the week column
week_df = week_df.reset_index()
week_df['week_rate'] = week_df['week_rate'].apply(lambda x: str(round(x*100,2)) + "%")

week_df.columns = ['week','Num_Camcom','Num_MV','week_rate']
# Print the resulting dataframe

week_rate = week_df.copy()


def week_period(week_number):
    week_number = int(week_number)
    year = 2023
    first_day = datetime.datetime(year,1,1)
    day_offset = (week_number -1)*7
    start_of_week = first_day + datetime.timedelta(days=day_offset)
    start_of_week = start_of_week.date()
    end_of_week = start_of_week + datetime.timedelta(days=6)
    
    
    return str(start_of_week) + "--" + str(end_of_week)

week_rate['week_period'] = week_rate['week'].apply(week_period)
week_rate = week_rate[['week_period','week','Num_Camcom','Num_MV','week_rate']]
week_rate

### Channel weekly adoption rate___df_partner_week

In [0]:
week_mv_portal = df_mv_portal[['Vehicle License Number','Type','month']].groupby(['month','Type']).nunique().reset_index()
week_mv_portal = week_mv_portal.ffill()
all_weeks = week_mv_portal['month'].unique().tolist()
week_mv_portal = week_mv_portal.set_index(['month','Type'])
#all_partner_type = ['Agent','Broker','Partner','Telemarketing']
all_partner_type = ['Agent','Partner']
multiindex = pd.MultiIndex.from_product([all_weeks,all_partner_type],names=['month','Type'])
week_mv_portal = week_mv_portal.reindex(multiindex,fill_value=0).reset_index()


w_camcom = week_camcom[['Inspection ID','Type','month']].groupby(['month','Type',]).nunique().reset_index()
w_camcom = w_camcom.ffill()
all_weeks = w_camcom['month'].unique().tolist()
w_camcom = w_camcom.set_index(['month','Type'])
all_partner_types = ['Agent','Partner']
multiindex = pd.MultiIndex.from_product([all_weeks,all_partner_type],names=['month','Type'])
w_camcom = w_camcom.reindex(multiindex,fill_value=0).reset_index()


# Merge the two dataframes on week
partner_week = pd.merge(w_camcom, week_mv_portal, on=['month','Type'])
partner_week ['month_rate'] = partner_week['Inspection ID'] / partner_week['Vehicle License Number']
#partner_week = partner_week.reset_index()
partner_week['month_rate'] = partner_week['month_rate'].apply(lambda x: str(round(x*100,2)) + "%")

partner_week.columns = ['month','Type','Num_Camcom','Num_MV','month_rate']
df_partner_month = partner_week.copy()
df_partner_month

In [0]:
week_mv_portal = df_mv_portal[['Vehicle License Number','Type','week']].groupby(['week','Type']).nunique().reset_index()
week_mv_portal = week_mv_portal.ffill()
all_weeks = week_mv_portal['week'].unique().tolist()
week_mv_portal = week_mv_portal.set_index(['week','Type'])
all_partner_type = ['Agent','Partner']
multiindex = pd.MultiIndex.from_product([all_weeks,all_partner_type],names=['week','Type'])
week_mv_portal = week_mv_portal.reindex(multiindex,fill_value=0).reset_index()


w_camcom = week_camcom[['Inspection ID','Type','week']].groupby(['week','Type',]).nunique().reset_index()
w_camcom = w_camcom.ffill()
all_weeks = w_camcom['week'].unique().tolist()
w_camcom = w_camcom.set_index(['week','Type'])
all_partner_types = ['Agent','Partner']
multiindex = pd.MultiIndex.from_product([all_weeks,all_partner_type],names=['week','Type'])
w_camcom = w_camcom.reindex(multiindex,fill_value=0).reset_index()


# Merge the two dataframes on week
partner_week = pd.merge(w_camcom, week_mv_portal, on=['week','Type'])
partner_week ['week_rate'] = partner_week['Inspection ID'] / partner_week['Vehicle License Number']
#partner_week = partner_week.reset_index()
partner_week['week_rate'] = partner_week['week_rate'].apply(lambda x: str(round(x*100,2)) + "%")

partner_week.columns = ['week','Type','Num_Camcom','Num_MV','week_rate']
df_partner_week = partner_week.copy()
df_partner_week

# Surveyors Performance

In [0]:

survey_job_s = pd.read_csv("xxxx")
survey_job_a = pd.read_csv("xxxx")
#survey_job_ali = survey_job_ali[~survey_job_ali['status'].isna()].reset_index().copy()



##A team

In [0]:
total_cases = survey_job_a[['Inspector','License Plate']].groupby(['Inspector'],as_index=False).size()
total_cases_status = survey_job_a[['Inspector','status','License Plate']].groupby(['Inspector','status'],as_index=False).size()
total_cases_status = total_cases_status.merge(total_cases, on = 'Inspector', how = 'left').copy()
total_cases_status.columns = ['Inspector','status','No for each status','Total No']
total_cases_status['Percentage'] = total_cases_status['No for each status'] / total_cases_status['Total No'] *100
total_cases_status['Percentage'] = total_cases_status['Percentage'].apply(lambda x: str(round(x,2)) + '%')
total_cases_status_total = total_cases_status.copy()

In [0]:
total_cases = survey_job_a[['Inspector','License Plate','month']].groupby(['Inspector','month'],as_index=False).size()
total_cases_status = survey_job_ali[['Inspector','status','License Plate','month']].groupby(['Inspector','status','month'],as_index=False).size()
total_cases_status = total_cases_status.merge(total_cases, on = ['Inspector','month'], how = 'left').copy()
total_cases_status = total_cases_status.sort_values(by=['Inspector', 'month'])
total_cases_status.columns = ['Inspector','status','month', 'No for each status','Total No of each month']
total_cases_status['Percentage'] = total_cases_status['No for each status'] / total_cases_status['Total No of each month'] *100
total_cases_status['Percentage'] = total_cases_status['Percentage'].apply(lambda x: str(round(x,2)) + '%')
total_cases_status_month = total_cases_status.copy()

In [0]:
import csv

# convert dataframes to lists of lists
data1 = [total_cases_status_total.columns.tolist()] + total_cases_status_total.values.tolist() # total adoption rate
data2 = [total_cases_status_month.columns.tolist()] + total_cases_status_month.values.tolist() # monthly adoption rate
data3 = [survey_job_ali.columns.tolist()] +survey_job_ali.values.tolist() # weekly adoption rate

# write data to CSV file
with open('"xxxx"', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data1)
    writer.writerows(['']*2)  # write two empty lines
    writer.writerows(data2)
    writer.writerows(['']*2) 
    writer.writerows(data3)
    writer.writerows(['']*2) 


##S team

In [0]:

survey_job_s = pd.read_csv("xxxx")
survey_job_a = pd.read_csv("xxxx")




In [0]:
total_cases = survey_job_s[['Inspector','License Plate']].groupby(['Inspector'],as_index=False).size()
total_cases_status = survey_job_s[['Inspector','status','License Plate']].groupby(['Inspector','status'],as_index=False).size()
total_cases_status = total_cases_status.merge(total_cases, on = 'Inspector', how = 'left').copy()
total_cases_status.columns = ['Inspector','status','No for each status','Total No']
total_cases_status['Percentage'] = total_cases_status['No for each status'] / total_cases_status['Total No'] *100
total_cases_status['Percentage'] = total_cases_status['Percentage'].apply(lambda x: str(round(x,2)) + '%')
total_cases_status_total = total_cases_status.copy()

In [0]:
total_cases = survey_job_s[['Inspector','License Plate','month']].groupby(['Inspector','month'],as_index=False).size()
total_cases_status = survey_job_s[['Inspector','status','License Plate','month']].groupby(['Inspector','status','month'],as_index=False).size()
total_cases_status = total_cases_status.merge(total_cases, on = ['Inspector','month'], how = 'left').copy()
total_cases_status = total_cases_status.sort_values(by=['Inspector', 'month'])
total_cases_status.columns = ['Inspector','status','month', 'No for each status','Total No of each month']
total_cases_status['Percentage'] = total_cases_status['No for each status'] / total_cases_status['Total No of each month'] *100
total_cases_status['Percentage'] = total_cases_status['Percentage'].apply(lambda x: str(round(x,2)) + '%')
total_cases_status_month = total_cases_status.copy()

In [0]:
import csv

# convert dataframes to lists of lists
data1 = [total_cases_status_total.columns.tolist()] + total_cases_status_total.values.tolist() # total adoption rate
data2 = [total_cases_status_month.columns.tolist()] + total_cases_status_month.values.tolist() # monthly adoption rate
data3 = [survey_job_shita.columns.tolist()] +survey_job_shita.values.tolist() # weekly adoption rate

# write data to CSV file
with open("xxxx"', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data1)
    writer.writerows(['']*2)  # write two empty lines
    writer.writerows(data2)
    writer.writerows(['']*2) 
    writer.writerows(data3)
    writer.writerows(['']*2) 


#CamCom Usage

In [0]:
camcom_usage_path = PATH1 + 'camcom-sftp/camcom-timestamp'
df_camcom_usage = pd.DataFrame()
for i in os.listdir(camcom_usage_path):
    try:
        df_temp = pd.read_csv(camcom_usage_path + '/'+i)
        df_camcom_usage = df_camcom_usage.append(df_temp, ignore_index = True)
    except:
        pass
df_camcom_usage = df_camcom_usage[~df_camcom_usage['FileName'].str.contains('completed')]
df_camcom_usage = df_camcom_usage[~df_camcom_usage['FileName'].str.contains('images')]
df_camcom_usage['SFTP_Time'] = pd.to_datetime(df_camcom_usage['SFTP_Time'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
df_camcom_usage['year'] = pd.to_datetime(df_camcom_usage['SFTP_Time']).apply(lambda x: x.strftime('%Y'))

df_camcom_usage['week'] = pd.to_datetime(df_camcom_usage['SFTP_Time']).apply(lambda x: x.strftime('%U'))
df_camcom_usage['month'] = pd.to_datetime(df_camcom_usage['SFTP_Time']).apply(lambda x: x.strftime('%m'))
df_camcom_usage = df_camcom_usage[df_camcom_usage['year']==str(2023)]
df_counts_month = df_camcom_usage.groupby('month').nunique()['FileName'].reset_index()
df_counts_week = df_camcom_usage.groupby('week').nunique()['FileName'].reset_index()
def week_period(week_number):
    week_number = int(week_number)
    year = 2023
    first_day = datetime.datetime(year,1,1)
    day_offset = (week_number -1)*7
    start_of_week = first_day + datetime.timedelta(days=day_offset)
    start_of_week = start_of_week.date()
    end_of_week = start_of_week + datetime.timedelta(days=6)
    
    
    return str(start_of_week) + "--" + str(end_of_week)

df_counts_week['week_period'] = df_counts_week['week'].apply(week_period)
import matplotlib.ticker as mtick
import io
import base64
import matplotlib.pyplot as plt
import mpld3


# Convert the week_rate column to a float
df_counts_week = df_counts_week.tail(15)

import matplotlib.pyplot as plt
import mpld3
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5),gridspec_kw={'wspace': 0.6})

axs[0].plot(df_counts_month['month'],df_counts_month['FileName'],marker='o')
axs[0].set_xticks(df_counts_month['month'])
axs[0].set_xticklabels(df_counts_month['month'],rotation=45)

# y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
# axs[0].yaxis.set_major_formatter(y_fmt)

positions = range(len(df_counts_month))
bar_width = 0.2
adj_positions = [pos + bar_width for pos in positions]
axs0 = axs[0].twinx()
# Plot the bar graph
axs0.bar(positions, df_counts_month['FileName'], color='#6495ED', width=bar_width, label='Num_Camcom')
#axs0.bar(adj_positions, month_rate['Num_MV'], color='blue', width=bar_width, label='Num_MV')
axs0.set_ylabel('Number')
# axs0.tick_params('y', colors='red')

axs[0].set_xlabel('Month')
axs[0].set_ylabel('Month Rate (as percentage)')
axs[0].set_title('Month Rate vs Month')
axs[0].legend()




axs[1].plot(df_counts_week['week_period'],df_counts_week['FileName'],marker='o')
axs[1].set_xticks(df_counts_week['week_period'])
axs[1].set_xticklabels(df_counts_week['week_period'],rotation=90)
# y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
# axs[1].yaxis.set_major_formatter(y_fmt)
positions = range(len(df_counts_week))
bar_width = 0.2
adj_positions = [pos + bar_width for pos in positions]
axs1 = axs[1].twinx()
# Plot the bar graph
axs1.bar(positions, df_counts_week['FileName'], color='#6495ED', width=bar_width, label='Num_Camcom')
#axs0.bar(adj_positions, month_rate['Num_MV'], color='blue', width=bar_width, label='Num_MV')
axs1.set_ylabel('Number')
# axs0.tick_params('y', colors='red')
axs[1].set_xlabel('Week Period')
axs[1].set_ylabel('Week Rate (as percentage)')
axs[1].set_title('Week Rate vs Week Period')
axs[1].legend()


buffer = io.BytesIO()
fig.savefig(buffer, format='png')
buffer.seek(0)
image_data = base64.b64encode(buffer.getvalue()).decode()
html_graph_camcom_usage = f'<img src="data:image/png;base64,{image_data}">'

# Graph

###Graph for adoption rate

In [0]:
week_df['week'] = week_df['week'].apply(lambda x: str(x).zfill(2))

current_week = datetime.datetime.now().strftime('%U')
if current_week not in week_df['week'].tolist():
    week_df = week_df.append({'week':str(current_week).zfill(2),'Num_Camcom':0,'Num_MV':0,'week_rate':"0%"},ignore_index=True)
for i in range(5, int(current_week)):
    if str(i).zfill(2) not in week_df['week'].tolist():
        week_df = week_df.append({'week':str(i).zfill(2),'Num_Camcom':0,'Num_MV':0,'week_rate':"0%"},ignore_index=True)


def week_period(week_number):
    week_number = int(week_number)
    year = 2023
    first_day = datetime.datetime(year,1,1)
    day_offset = (week_number -1)*7
    start_of_week = first_day + datetime.timedelta(days=day_offset)
    start_of_week = start_of_week.date()
    end_of_week = start_of_week + datetime.timedelta(days=6)
    
    
    return str(start_of_week) + "--" + str(end_of_week)

week_df['week_period'] = week_df['week'].apply(week_period)
week_df = week_df.sort_values(by='week')

partner_week['week_period'] = partner_week['week'].apply(week_period)


In [0]:
import matplotlib.ticker as mtick
import io
import base64
import matplotlib.pyplot as plt
import mpld3


# Convert the week_rate column to a float
week_df1 = week_df.tail(9)
month_rate['month_rate'] = month_rate['month_rate'].str.rstrip('%').astype('float') / 100
week_df1['week_rate'] = week_df1['week_rate'].str.rstrip('%').astype('float') / 100

import matplotlib.pyplot as plt
import mpld3
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5),gridspec_kw={'wspace': 0.6})

axs[0].plot(month_rate['month'],month_rate['month_rate'],marker='o')
axs[0].set_xticks(month_rate['month'])
axs[0].set_xticklabels(month_rate['month'],rotation=45)
y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
axs[0].yaxis.set_major_formatter(y_fmt)

positions = range(len(month_rate))
bar_width = 0.2
adj_positions = [pos + bar_width for pos in positions]
axs0 = axs[0].twinx()
# Plot the bar graph
axs0.bar(positions, month_rate['Num_Camcom'], color='#6495ED', width=bar_width, label='Num_Camcom')
#axs0.bar(adj_positions, month_rate['Num_MV'], color='blue', width=bar_width, label='Num_MV')
axs0.set_ylabel('Number')
# axs0.tick_params('y', colors='red')

axs[0].set_xlabel('Month')
axs[0].set_ylabel('Month Rate (as percentage)')
axs[0].set_title('Month Rate vs Month')
axs[0].legend()




axs[1].plot(week_df1['week_period'],week_df1['week_rate'],marker='o')
axs[1].set_xticks(week_df1['week_period'])
axs[1].set_xticklabels(week_df1['week_period'],rotation=90)
y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
axs[1].yaxis.set_major_formatter(y_fmt)
positions = range(len(week_df1))
bar_width = 0.2
adj_positions = [pos + bar_width for pos in positions]
axs1 = axs[1].twinx()
# Plot the bar graph

axs1.bar(positions, week_df1['Num_Camcom'], color='#6495ED', width=bar_width, label='Num_Camcom')
#axs0.bar(adj_positions, month_rate['Num_MV'], color='blue', width=bar_width, label='Num_MV')
axs1.set_ylabel('Number')
# axs0.tick_params('y', colors='red')
axs[1].set_xlabel('Week Period')
axs[1].set_ylabel('Week Rate (as percentage)')
axs[1].set_title('Week Rate vs Week Period')
axs[1].legend()


buffer = io.BytesIO()
fig.savefig(buffer, format='png')
buffer.seek(0)
image_data = base64.b64encode(buffer.getvalue()).decode()
html_graph = f'<img src="data:image/png;base64,{image_data}">'

In [0]:


df_partner_month['month_rate'] = df_partner_month['month_rate'].str.rstrip('%').astype('float') / 100

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

    
for i, partner in enumerate(df_partner_month['Type'].unique()):
    data = df_partner_month[df_partner_month['Type'] == partner]
    if i == 0:
        axs[0].plot(data['month'], data['month_rate'], color='blue', label=partner,marker='o')
    else:
        
        axs[0].plot(data['month'], data['month_rate'], color='green', label=partner,marker='o')
    # elif i == 2:
    #     axs[0].plot(data['month'], data['month_rate'], color='orange', label=partner,marker='o')
    # else:
    #     axs[0].plot(data['month'], data['month_rate'], color='#8B5A2B', label=partner,marker='o') # Light Brown

        
# Add labels and legend
axs[0].set_xlabel('month')
axs[0].set_xticklabels(df_partner_month['month'].unique())
#ax.set_xticklabels(week_df['week_period'],rotation=45)
y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
axs[0].yaxis.set_major_formatter(y_fmt)
axs[0].set_ylabel('Monthly Rate')
axs[0].legend(loc='upper left')


partner_week['week_rate'] = partner_week['week_rate'].str.rstrip('%').astype('float') / 100
    
for i, partner in enumerate(partner_week['Type'].unique()):
    data = partner_week[partner_week['Type'] == partner]
    if i == 0:
        axs[1].plot(data['week_period'], data['week_rate'], color='blue', label=partner,marker='o')
    else:
        
        axs[1].plot(data['week_period'], data['week_rate'], color='green', label=partner,marker='o')
    # elif i == 2:
    #     axs[1].plot(data['week_period'], data['week_rate'], color='orange', label=partner,marker='o')
    # else:
    #     axs[1].plot(data['week_period'], data['week_rate'], color='#8B5A2B', label=partner,marker='o') # Light Brown

        
# Add labels and legend
axs[1].set_xlabel('week_period')
axs[1].set_xticklabels(partner_week['week_period'].unique(),rotation=90)
#ax.set_xticklabels(week_df['week_period'],rotation=45)
y_fmt = mtick.PercentFormatter(xmax=1.0, decimals=2)
axs[1].yaxis.set_major_formatter(y_fmt)
axs[1].set_ylabel('Week Rate')
axs[1].legend(loc='upper left')


# Show plot
plt.show()

buffer = io.BytesIO()
fig.savefig(buffer, format='png')
buffer.seek(0)
image_data = base64.b64encode(buffer.getvalue()).decode()
html_graph_channel = f'<img src="data:image/png;base64,{image_data}">'



### Graph for surveyor performance

In [0]:

survey_job_s = pd.read_csv("xxxx"')
survey_job_a = pd.read_csv("xxxx")
survey_job_a = survey_job_a[~survey_job_a['status'].isna()].reset_index().copy()
survey_job_a = survey_job_a[survey_job_a['status'] == 'review_completed']



In [0]:
avg_unqualified_images_a = survey_job_a[['License Plate','Inspector','No of Failed Images','week','month']].copy()
avg_unqualified_images_a = avg_unqualified_images_a[avg_unqualified_images_a['week']>=5]
week_avg_unqualified_images_a = avg_unqualified_images_a[['Inspector','week','No of Failed Images']].groupby(['Inspector','week']).mean().reset_index()
week_avg_unqualified_images_a = week_avg_unqualified_images_a.ffill()


current_week = datetime.datetime.now().strftime('%U')
week_avg_unqualified_images_1= pd.DataFrame()
for i, inspector in enumerate(week_avg_unqualified_images_a['Inspector'].unique()):

    data = week_avg_unqualified_images_a[week_avg_unqualified_images_a['Inspector'] == inspector]

    if int(current_week) not in data['week'].tolist():
        data=data.append({'week':str(current_week).zfill(2),'No of Failed Images':0,'Inspector':inspector},ignore_index=True)

    for i in range(5, int(current_week)):
        if i not in data['week'].tolist():
            data = data.append({'week':str(i).zfill(2),'No of Failed Images':0,'Inspector':inspector},ignore_index=True)
            data.sort_values(by=['Inspector','week'],inplace=True)
    
        
    week_avg_unqualified_images_1 = pd.concat([week_avg_unqualified_images_1,data])
week_avg_unqualified_images_1['week'] = week_avg_unqualified_images_1['week'].astype(int)
week_avg_unqualified_images_1 = week_avg_unqualified_images_1.sort_values(by=['Inspector','week'])
week_avg_unqualified_images_1['week_period'] = week_avg_unqualified_images_1['week'].apply(week_period)


import matplotlib.ticker as mtick
import io
import base64
import matplotlib.pyplot as plt
import mpld3
fig, ax = plt.subplots(figsize=(8, 6))

week_avg_unqualified_images_1 = week_avg_unqualified_images_1[week_avg_unqualified_images_1['week'].astype(int) >= 5]
    
for i, inspector in enumerate(week_avg_unqualified_images_1['Inspector'].unique()):
    data = week_avg_unqualified_images_1[week_avg_unqualified_images_1['Inspector'] == inspector]
    if i == 0:
        ax.plot(data['week_period'], data['No of Failed Images'], color='blue', label=inspector,marker='o')
    elif i == 1:
        data.sort_values(by='week_period',inplace=True)
        ax.plot(data['week_period'], data['No of Failed Images'], color='green', label=inspector,marker='o')

    elif i == 2:
        ax.plot(data['week_period'], data['No of Failed Images'], color='orange', label=inspector,marker='o')
    else:
        ax.plot(data['week_period'], data['No of Failed Images'], color='#8B5A2B', label=inspector,marker='o') # Light Brown


        
# Add labels and legend

ax.set_xlabel('week_period')
ax.set_xticklabels(week_avg_unqualified_images_1[week_avg_unqualified_images_1['Inspector'] == 'Inspector 2 - Edi Kartono - 082198508597']['week_period'],rotation=90)
#ax.set_xticklabels(week_df['week_period'],rotation=45)
ax.set_ylabel('Avg week num of unqualified images')
ax.legend(loc='upper left')
ax.set_title('Ali Team Surveyors Performance')

# Show plot
plt.show()

buffer = io.BytesIO()
fig.savefig(buffer, format='png')
buffer.seek(0)
image_data = base64.b64encode(buffer.getvalue()).decode()
html_graph_unqualified_ali = f'<img src="data:image/png;base64,{image_data}">'

In [0]:
avg_unqualified_images_shita = survey_job_shita[['License Plate','Inspector','No of Failed Images','week','month']].copy()
avg_unqualified_images_shita = avg_unqualified_images_shita[avg_unqualified_images_shita['week']>=23]

grouped = avg_unqualified_images_shita[['Inspector','No of Failed Images']].groupby('Inspector')['No of Failed Images'].mean().sort_values(ascending=False).reset_index()
top_inspectors = grouped.head(4)
avg_unqualified_images_shita = avg_unqualified_images_shita[avg_unqualified_images_shita['Inspector'].isin(list(top_inspectors['Inspector']))]

week_avg_unqualified_images_shita = avg_unqualified_images_shita[['Inspector','week','No of Failed Images']].groupby(['Inspector','week']).mean().reset_index()
week_avg_unqualified_images_shita = week_avg_unqualified_images_shita.ffill()


current_week = datetime.datetime.now().strftime('%U')
week_avg_unqualified_images_1= pd.DataFrame()
for i, inspector in enumerate(week_avg_unqualified_images_shita['Inspector'].unique()):

    data = week_avg_unqualified_images_shita[week_avg_unqualified_images_shita['Inspector'] == inspector]

    if int(current_week) not in data['week'].tolist():
        data=data.append({'week':str(current_week).zfill(2),'No of Failed Images':0,'Inspector':inspector},ignore_index=True)

    for i in range(23, int(current_week)):
        if i not in data['week'].tolist():
            data = data.append({'week':str(i).zfill(2),'No of Failed Images':0,'Inspector':inspector},ignore_index=True)
            data.sort_values(by=['Inspector','week'],inplace=True)
            print(data)
    
        
    week_avg_unqualified_images_1 = pd.concat([week_avg_unqualified_images_1,data])
week_avg_unqualified_images_1['week'] = week_avg_unqualified_images_1['week'].astype(int)
week_avg_unqualified_images_1 = week_avg_unqualified_images_1.sort_values(by=['Inspector','week'])
week_avg_unqualified_images_1['week_period'] = week_avg_unqualified_images_1['week'].apply(week_period)


import matplotlib.ticker as mtick
import io
import base64
import matplotlib.pyplot as plt
import mpld3
fig, ax = plt.subplots(figsize=(8, 6))

week_avg_unqualified_images_1 = week_avg_unqualified_images_1[week_avg_unqualified_images_1['week'].astype(int) >= 23]
    
for i, inspector in enumerate(week_avg_unqualified_images_1['Inspector'].unique()):
    data = week_avg_unqualified_images_1[week_avg_unqualified_images_1['Inspector'] == inspector]
    if i == 0:
        ax.plot(data['week_period'], data['No of Failed Images'], color='blue', label=inspector,marker='o')
    elif i == 1:
        data.sort_values(by='week_period',inplace=True)
        ax.plot(data['week_period'], data['No of Failed Images'], color='green', label=inspector,marker='o')

    elif i == 2:
        ax.plot(data['week_period'], data['No of Failed Images'], color='orange', label=inspector,marker='o')
    else:
        ax.plot(data['week_period'], data['No of Failed Images'], color='#8B5A2B', label=inspector,marker='o') # Light Brown


        
# Add labels and legend

ax.set_xlabel('week_period')
ax.set_xticklabels(week_avg_unqualified_images_1[week_avg_unqualified_images_1['Inspector'] == list(top_inspectors['Inspector'])[0]]['week_period'],rotation=90)
#ax.set_xticklabels(week_df['week_period'],rotation=45)
ax.set_ylabel('Avg week num of unqualified images')
ax.legend(loc='upper left')
ax.set_title('Shita Team Surveyors Performance')

# Show plot
plt.show()

buffer = io.BytesIO()
fig.savefig(buffer, format='png')
buffer.seek(0)
image_data = base64.b64encode(buffer.getvalue()).decode()
html_graph_unqualified_shita = f'<img src="data:image/png;base64,{image_data}">'

In [0]:
message_1 = '''
<head>
    <title>Error KPIs CamCom</title>
    <style>
      table,
      th,
      td {{
        padding: 10px;
        border: 1px solid black;
        border-collapse: collapse;
      }}
    </style>
</head>
'''
end_txt = '''<p> For further details on unqualified images and missed sides, please view attachment </p>'''


In [0]:
# convert the dataframe to html
# html_task = task_to_check.to_html()
# task_to_check = '''
# <p>This is the tasks we need to follow up, please check the following table:</p><br>
# '''

html_table1 = Adoption_Rate.to_html()
html_table5 = month_df.to_html()
html_table4 = week_rate.to_html()
new_df_html = new_df.to_html()

## zero_rate
Start_text = '''
<p>Hi Team:</p>
'''
Start_text_1 = '''
<p>An update for the adoption rate and surveyers perfotmance. Please review following tables and graphs.</p>
'''

zero_rate_text = '''
<p>This is the total rate for the TAT < 1day :</p><br>
'''
zero_rate = "{:.2f}%".format(zero_rate * 100)
zero_rate = "<p>The formatted percentage is: <strong>{}</strong></p>".format(zero_rate)
zero_rate_month_text = '''
<p>This is the monthly rate for the TAT<1day:</p><br>
'''


additional_text_1 = '''
<p>This is the monthly adoption rate graph (only track agent&partner at this stage):</p><br>
The calculation of adoption rate: b/a<br>
a = Total number of USED cars in MV_Portal Report from Feb 2023<br>
b = Total number of cases can be found in Camcom Protal<br>
'''
additional_text_2 = '''
<p>This is the surveyors performance with the number of failed cases and also the average weekly unqualified images:</p><br>
'''
additional_text_3 = '''
<br>
'''
additional_text_4 = '''
<p>This is the weekly adoption rate:</p><br>
'''
additional_text_5 = '''
<p>For more details,pls refer to the table below:</p><br>
'''
additional_text_6 = '''
<p>Following graph is the usage of CamCom.</p><br>'''

# final_html = Start_text+Start_text_1 + additional_text_1 + html_graph + html_graph_channel+ additional_text_5 +html_table5  + additional_text_4 + html_table4  + zero_rate_text+zero_rate +zero_rate_month_text+new_df_html+additional_text_2+ html_table2 +html_graph_unqualified + end_txt

final_html = Start_text+Start_text_1 + additional_text_1 + html_graph + html_graph_channel+additional_text_6+html_graph_camcom_usage +additional_text_5 +html_table5  + additional_text_4 + html_table4  + zero_rate_text+zero_rate +zero_rate_month_text+new_df_html +additional_text_2+ html_graph_unqualified_ali +html_graph_unqualified_shita + end_txt


#final_html = task_to_check + html_task

# print the final HTML code
print(final_html)


In [0]:
text_file = open("/dbfs/mnt/idpdmccpstrappdataz01/errors/sample.txt", "w")
n = text_file.write(final_html)
text_file.close()